In [703]:
import os
import regex
import shutil
import random
from tqdm import tqdm

In [704]:
# from txt_to_dataset import find_characters
# from txt_to_dataset import find_author
# from txt_to_dataset import find_end

---

In [705]:
TXT_DIR = "theatregratuit-txt"
off = "\t"
if os.path.isdir(TXT_DIR):
    fnames = [x for x in os.listdir(TXT_DIR) if ".txt" == os.path.splitext(x)[-1]] # don't load vim *.swp files
    n_files = len(fnames)

In [706]:
TMP_DIR = "tmp1"
if not os.path.isdir(TMP_DIR):
    os.mkdir(TMP_DIR)

---

In [707]:
def printlines(lines):
    print('\n'.join(lines))

In [708]:
def printsep(s):
    print(s*40)

In [709]:
def underprint(x):
    print(x)
    print('-'*len(x))

In [710]:
def print_index_split(lines, index):
    start = index - 10
    if start < 0: start = 0
    printlines(lines[start:index])
    printsep('*')
    printlines(lines[index:index+10])

In [711]:
def print_file(f, whole_file=False):
    underprint(f)
    print()
    raw, lines, lines_len = get_lines(f)
    if whole_file:
        printlines(lines)
    else:
        lines_shortened, _ = shortened_lines(f, lines, lines_len)
        printlines(lines_shortened)

In [712]:
def print_random_file(files, whole_file=False):
    total = len(files)
    ind = random.randint(0, total-1)
    f = files[ind]
    print(f"file {ind}/{total}")
    print_file(f, whole_file=whole_file)

---

In [713]:
def get_lines(input_name):
    with open(os.path.join(TXT_DIR, input_name), "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read()
        lines = [l.strip() for l in raw.split('\n')]
    return raw, lines, len(lines)

In [714]:
def shortened_lines(fname, lines, lines_len):
    author_index = find_author(fname, lines, lines_len) 
    if not author_index:
        author_index = index_of_regex_match(lines, REGICES["additional_author"])
    char_index = find_characters(fname, lines, lines_len)        
    end_index = find_end(fname, lines, lines_len)
    # print(fname, char_index, author_index, end_index)
    start_index = max(author_index, char_index)
    return lines[start_index:end_index], {
        "char_index": char_index, 
        "author_index": author_index, 
        "end_index": end_index,
        "start_index": start_index
    }

In [715]:
def end_of_block_and_trim(lines, lines_len, j, k):
    # find end of block
    while j+k < lines_len and not regex.match(REGICES["blank_line"], lines[j + k]):
        k += 1
    # trim empty lines
    while j+k < lines_len and regex.match(REGICES["blank_line"], lines[j + k]):
        k += 1     
    return k

In [733]:
def find_characters(fname, lines, lines_len):  
    feydeau = False # almost all Feydeau files have 2-3 blocks of char
    vega = False # Vega files can be cut up to line "La scène..."

    char_index = 0
    found_char = False
    limit = 15
    
    if fname in (
        "count-1537913-LES_ESPAGNOLS_EN_DANEMARK.txt",
        "count-1712853-LE_VERITABLE_SAINT_GENEST.txt",
        "count-3263112-LA_MORT_DE_WALLENSTEIN.txt",
        "count-2267304-LAmour_medecin.txt",
        "count-2037607-LHabit_vert.txt", 
        "count-2158361-ANDROMEDE.txt",
        "count-3109664-SYLVANIRE.txt"
        "count-3397657-LA_VEINE.txt",
        "count-2047890-LE_ROI.txt",
        "count-3031455-Nono.txt",
    ):
        feydeau = True
    
    # have a long prologue
    if fname in ("count-1355612-UNE_FEMME_EST_UN_DIABLE.txt"
                ):
        limit = 27

    for j, l in enumerate(lines[:limit]):
        if found_char:
            break
        if "Feydeau" in l and fname not in (
            "count-2199212-Les_Paves_de_lours.txt",
            "count-2316496-On_va_faire_la_cocotte.txt",            
        ):
            feydeau = True
        if "Lope de Vega" in l:
            vega = True
        if regex.match(REGICES["character"], l):
            found_char = True
            k = 1
            # files with space after char > k + 1
            if fname in (
                "count-1072543-CornPR.txt"                
                "count-2158361-ANDROMEDE.txt",
                "count-2037607-LHabit_vert.txt",
                "count-1182649-LE_MONDE_OU_L.txt",
                "count-958980-Le_gendre_de_Monsieur_Poirier_-_Emile_Augier.txt",                
                "count-1037209-LES_MAMELLES_DE_TIRESIAS_-_Guillaume_Apollinaire.txt",
            ):
                k += 1
            # more annoying exceptions
            if fname in ("count-959005-LOrphelin_de_la_Chine_-_Voltaire.txt",):
                # find init caps words only, no space (lest we go to the end)
                while j+k < lines_len and regex.match(REGICES["WORD"], lines[j + k]):
                    k += 1    
            elif vega: 
                # all vega can go up to "La scène"
                while j+k < lines_len and not regex.match(REGICES["la_scene"], lines[j + k]):
                    k += 1                
            else:
                k = end_of_block_and_trim(lines, lines_len, j, k)
                # annoying exceptions: more chars after blank line
                if feydeau:
                    k = end_of_block_and_trim(lines, lines_len, j, k)
                    # some files like Feydeau with three !
                    if fname in ("count-1537913-LES_ESPAGNOLS_EN_DANEMARK.txt",
                                 "count-3263112-LA_MORT_DE_WALLENSTEIN.txt",
                                 "count-2267304-LAmour_medecin.txt", 
                                 "count-3397657-LA_VEINE.txt"):
                        k = end_of_block_and_trim(lines, lines_len, j, k)
            char_index = j + k          
    return char_index

In [734]:
def find_author(fname, lines, lines_len):
    author_index = 0
    found_author = False
    for j, l in enumerate(lines[:10]):
        if found_author:
            break
        if regex.match(REGICES["author"], lines[j]):
            found_author = True
            # annoying exceptions
            if fname in ("count-1049195-Arret_36_de_lautobus_40_-_Jean_Sibil.txt",
                         "count-1396213-Roberto_Succo.txt"):  
                k = 1
            elif fname == "count-1541294-Olaf_loriginal.txt":
                k = 2 
            else:
                k = 2  # we assume "de\n\nauthor"
                while j + k < lines_len and not regex.match(
                    REGICES["blank_line"], lines[j + k]
                ):
                    k += 1
            author_index = j + k + 1
    return author_index

In [718]:
def find_end(fname, lines, lines_len):
    end_index = lines_len - 1
    found_end = False
    for j, l in enumerate(reversed(lines[-4:])):
        if found_end:
            break
        if regex.search(REGICES["fin"], l):
            found_end = True
            end_index = end_index - j - 1
    return end_index

In [719]:
def index_of_regex_match(lines, r, trim=True):
    lines_len = len(lines)
    ind = 0
    found = False
    for i,l in enumerate(lines):
        if found: break
        if regex.search(r, l):
            found = True
            k = 1            
            if trim: 
                while i + k < lines_len and \
                 regex.match(REGICES["blank_line"], lines[i + k]):
                    k += 1
            ind = i + k
            break
    return ind 

In [720]:
def file_data(f, verbose=True):
    if verbose:
        underprint(f)
        print()
    raw, lines, lines_len = get_lines(f)
    lines_shortened, indices = shortened_lines(f, lines, lines_len)
    lines_shortened_len = len(lines_shortened)
    data = {
        "fname": f,
        "raw": raw,
        "lines": lines,
        "lines_len": lines_len,
        "lines_shortened": lines_shortened,
        "lines_shortened_len": lines_shortened_len
    }
    data.update(indices)
    return data

In [721]:
def make_regices():
    return {
        "blank_line": regex.compile("^\s*$"),
        "blank_line_with_rubbish": regex.compile("^[\p{Z}\p{P}]*$"),
        
        # trim beginning & end of files
        # -----------------------------
        "character": regex.compile(
            "^((les\s*)*pe*rsonna *ge|(les)*acteurs|dramatis personae|avertissement|entreparleur|biographies|apparences|personrage|persongueules|pépersonâge)",
            regex.IGNORECASE,
            ),
        "author": regex.compile("^de$",regex.IGNORECASE),
        "additional_author": regex.compile("^(\(Auteur inconnu\)|" +\
                                           "Voltaire|" +\
                                           "de\s+Georges Courteline|" +\
                                           "Translation en prose de Jean Sibil|" +\
                                           "Anton Pavlovitch Tchekhov|" +\
                                           "EUGENE LABICHE, A. LEFRANC et MARC-MICHEL|" +\
                                           "Alexandre Hardy)$"),
        "la_scene": regex.compile("^La scène"),        
        "fin": regex.compile("(fin|rideau|f1n|inachev|manque)", regex.IGNORECASE),
        
        
        # search for lines made of full caps + punct/space, as well as some
        # very common words listing characters, and didascalia in ()
        "caps_full": regex.compile("^\s*((\p{Lu}+|seule?|puis|puis tout le monde|moins|et)\s*[.’'<>;,]*\s*)+\s*(\(.*?\))*\s*$"),
        # common annoying starts "M.", "L'"
        "annoying_init": regex.compile("^[\p{P}\p{Z}]*(M\.|\p{Lu}')\p{Z}*"),
        # words (with possible - or '), and space, with punctuation at the end
        "caps_init": regex.compile("^\s*(([\p{Lu}\p{Pc}\p{Pd}1]+\p{Zs}*)+)(\s*[\p{Po}]\s*)"),
        "char_lc_dot_dash": regex.compile("^(\p{Z}*M*[\p{Ll}\p{Z}]+\.)\p{Z}*\p{Pd}+\p{Z}*(.*)$"),
        "trailing_space": regex.compile("\s*$"),
        "trailing_punct": regex.compile("\s*[.,:]*\s+$"),
        "non_breaking_space": regex.compile(" {2,}"), # non-breaking space
        "M." : regex.compile("M.\s"),
        "dot": regex.compile("\s*\.\s*"),
        "final_dot": regex.compile("\s*\.\s*$"),
        "final_comma": regex.compile(",$"),
        
        # include rare errors like ".-—Blah" or ". -— Blah"
        "dash": regex.compile("[,.](\s*[-–—]+|-[–—])\s*"),
        "dash_and_more": regex.compile("(.*?)[.;:,]*\s*-*[-–—]\s*"),
        "colon": regex.compile("\s*:\s*"),
        "colon_and_more": regex.compile("(.*?)\s*:\s*"),
        "didasc_and_more": regex.compile(".*?(?<!M)[.:]\s*"),
        # "char_no_dot": regex.compile("^\s*([A-Z1'-]+)\s*$"),
        # # using all 3 lengths: -, –, —, bc of inconsistencies
        # "char_dash": regex.compile("^\s*([A-Z1'-]{2,})\.[\s-]+[–—]\s"),
        # "char_comma_dash": regex.compile("^\s*([A-Z1'-]{2,},.*?)\.[\s-]+[–—]\s"),
        # "char_dot_more": regex.compile("^\s*([A-Z1'-]{2,}\.)(.*)$"),
        # "char_no_dot_more": regex.compile("^\s*([A-Z1'-]{2,})(.*)$"),
        
        # regices for corpus splitting
        # ----------------------------
        "WORD": regex.compile("\p{Lu}{2,}"), 
        "WORD_INIT": regex.compile("^((L')*\p{Lu}{2,}|[AB]\.)"),         
        "CHAR_ONELINE": regex.compile("^\p{Lu}+[\p{Z}\p{P}]*$"),
        "CHAR_ONELINE_DOT": regex.compile("^\p{Lu}+\p{Z}*\.\p{Z}*$"),
        "CHAR_ONELINE_COLON": regex.compile("^\p{Lu}+\p{Z}*:\p{Z}*$"),
        "INIT_char_DOT_n_DASH" : regex.compile("^\p{L}+\.\p{Z}+\p{Pd}"),
        "INIT_char_COLON" : regex.compile("^\p{L}+\p{Z}+:"),          
        "INIT_CHAR_DOT_N_DASH" : regex.compile("^\p{Lu}+\.\p{Z}+\p{Pd}"),
        "INIT_CHAR_COLON" : regex.compile("^\p{Lu}+\p{Z}+:"),      
        "init_char_dot_n_dash" : regex.compile("^\p{Ll}+\.\p{Z}+\p{Pd}"),
        "init_char_colon" : regex.compile("^\p{Ll}+\p{Z}+:"),
        
        
        # regices for innards cleaning
        # ----------------------------
        # letters & space, possibly a parenthesis, : or — at the end
        "char_colon_or_dash": regex.compile("^((?:L'|M\.)*[\p{L}\p{Pd}\p{Z}]+)" +\
                                            "(\p{Z}\(.*?\))*" +\
                                            "\p{Z}[:—]\p{Z}"),
        
        # lower case character, optionally with didasc, ending in dot and dash, 
        "char_lc_dot_dash": regex.compile("^([\p{Ll}\p{Z}]+)" +\
                                          "(([,\p{Z}].*?)*)" + \
                                          "\.\p{Z}*\p{Pd}\p{Z}*"),
        
        # char (with numbers & spaces) followed or not by :
        "char_oneline_colon": regex.compile("^((L')*[\p{L}\p{N}\p{Z}]+?)\p{Z}*:$"),
        
        "char_colon": regex.compile("^(\p{L}{2,})\p{Z}*:\p{Z}*"),
        # "char_colon_more": regex.compile("^\s*([A-Z1'-]{2,})\s*:\s*(.*)$"),
        # "first_scene": regex.compile("(sc[èe]ne (premi[èe]re|I)|premier tableau|I, 1\.)", regex.IGNORECASE),
        # "act_or_scene": regex.compile("(acte|sc[èe]ne|^I\s*$)", regex.IGNORECASE),
        
        # block (multiline) regices (to act on raw text)
        # "characters_block": regex.compile("\n*\s*PERSONNAGES?(.*?)(\n\s*\n)+", regex.DOTALL),
        
        # line cleanup
        "sc_ene": regex.compile("SC\p{Z}+[EÈ]NE", regex.IGNORECASE),
        "deuxi_eme": regex.compile("DEUXI\p{Z}ÈME", regex.IGNORECASE)
    }

REGICES = make_regices()

---
## All files.

In [722]:
fnames_it = iter(fnames)
ind = 0

In [723]:
fname = next(fnames_it)
# fname = "count-1396213-Roberto_Succo.txt"
ind +=1 
underprint(fname)
print(ind)
print()
raw, lines, lines_len = get_lines(fname)
# printlines(lines)
lines_shortened, data = shortened_lines(fname, lines, lines_len)
# printlines(lines_shortened)
print_index_split(lines, max(data["char_index"], data["author_index"]))

count-2454059-Champignol_malgre_lui.txt
---------------------------------------
1

LE BRIGADIER DE GENDARMERIE
ROUCHE, territorial
UN PERRUQUIER
UN CAPORAL

ANGELE, femme de CHAMPIGNOL
MAURICETTE, fille de CHAMEL
ADRIENNE, fille de CAMARET
CHARLOTTE

****************************************
Le 1er acte à Paris dans l'hôtel de CHAMPIGNOL.
Le 2* acte à Clermont.
Le 3* acte chez Mme Rivolet, dans les environs de Clermont.

ACTE I
A droite la chambre de madame CHAMPIGNOL. — A gauche, premier plan, porte donnant sur les appartements. — Au deuxième plan, grande fenêtre, porte au fond, donnant sur l'antichambre. — Au fond de l'antichambre, porte donnant sur l'escalier. — Tableaux sur des chevalets. — Etudes sur les murs, etc. - A droite, premier plan, une table flanquée de deux chaises, sur la table une tasse de chocolat servie; à gauche, un canapé, et, à côté du canapé et à droite, deux chaises volantes. — Au fond contre le mur, une toile placée sur une chaise et retournée.
SCENE PREMIERE
SA

# Trimming incipits/endings done, save intermediary state.

In [735]:
d = file_data("count-1537913-LES_ESPAGNOLS_EN_DANEMARK.txt")

count-1537913-LES_ESPAGNOLS_EN_DANEMARK.txt
-------------------------------------------



In [736]:
printlines(d["lines_shortened"])

La loge de Clara Gazul.
UN GRAND, UN CAPITAINE, UN POÈTE, CLARA

LE GRAND : Enfin vous êtes habillée !
LE POETE : Et toujours jolie comme un ange.
LE CAPITAINE : Eh quoi ! sans basquina et sans mantilla ?
CLARA : C'est que je n'ai pas à jouer un rôle espagnol.
LE CAPITAINE : Tant pis !
LE GRAND : Qu'est-ce que l'auteur ?
CLARA : Je ne sais.
LE POETE : Toujours discrète ! Ah ! que nous vous avons d'obligations, nous autres pauvres auteurs !
Ils s'asseyent tous.
CLARA : Voilà qui est bien, messieurs ! Vous vous asseyez ici, comme si vous aviez envie de passer la soirée dans cette loge. — Excellentissime seigneur, si vous vous mettez dans un fauteuil, vous allez vous endormir et manquer la comédie.
LE GRAND : Vous savez bien que je ne viens jamais qu'à la seconde journée.
LE POETE : Oh ! j'espère que la pièce nouvelle est divisée en actes.
CLARA : C'est ce qui vous trompe. Mais la comédie en restera-t-elle plus mauvaise ?
LE POETE : Hé ! elle n'en devient pas meilleure. D'abord le titre n

In [737]:
def save_trimmed_files(fnames):
    for f in tqdm(fnames):
        data = file_data(f, verbose=False)
        with open(os.path.join(TMP_DIR, f), 'w') as o:
            o.write('\n'.join(data["lines_shortened"]))

In [738]:
save_trimmed_files(fnames)

100%|██████████| 1083/1083 [00:01<00:00, 564.63it/s]


In [636]:
if os.path.isdir(TXT_DIR):
    ffnames = [x for x in os.listdir(TMP_DIR) if ".txt" == os.path.splitext(x)[-1]]  # don't load vim *.swp files
    n_ffiles = len(fnames)

In [637]:
print_random_file(ffnames)

file 164/1083
count-2316483-La_Femme_qui_perd_ses_jarretieres.txt
---------------------------------------------------

La scène est à Paris.

Un salon octogone. Porte au fond, portes latérales, une fenêtre à droite dans l'angle. A l'angle de gauche, un grand portrait d'homme accroché au mur; sous le portrait, une console; guéridon à droite au premier plan; table à gauche, au premier plan.

SCÈNE PREMIÈRE
LAVERDURE, seul. Il est en habit de livrée, une brosse au pied et un balai à la main.
Quelqu'un qui me verrait ainsi, en habit de livrée, en train de frotter... dirait bien certainement : Voilà un domestique qui fait l'appartement de son maître... Eh bien! cette dame se tromperait... Cet appartement est le mien, ce mobilier est à moi, ce balai est ma propriété... Je ne suis pas domestique... je n'ai pas de maître... Je vis de mes rentes! J'ai servi dix ans... pas comme militaire... comme valet de chambre... un Anglais puissamment riche, mais qui avait des maux d'estomac! (Montrant le p

---
## All the trimming...

---
### Ending

Unproblematic (**hopefully**). `find_end` does the job, used in `shortened_lines`.

In [179]:
with_end, without_end = split_by_regex(fnames, REGICES["fin"])

with: 1083
without: 0


---
### Author

Split by formatted author: clean author files with formatted author name and files without.

In [352]:
auth_fname, no_auth_fnames = split_by_regex(fnames, REGICES["author"], orig_file=True)

with: 1070
without: 13


In [353]:
fnames_it = iter(no_auth_fnames)

In [364]:
fname = next(fnames_it)
raw, lines, lines_len = get_lines(fname)
author_index = index_of_regex_match(lines, REGICES["additional_author"], trim=True)
print_index_split(lines, author_index)


****************************************
Ruy Blas

1858

Personnages

RUY BLAS
DON SALLUSTE DE BAZAN
DON CÉSAR DE BAZAN
DON GURITAN


In [365]:
fnames_it = iter(auth_fname)

In [381]:
fname = next(fnames_it)
raw, lines, lines_len = get_lines(fname)
author_index = find_author(fname, lines, lines_len)
print_index_split(lines, author_index)

La Punaise

de

Vladimir Maïakovski

****************************************
Traduction de M. Wassiltchikov

PERSONNAGES :
PRISSIPKINE, PIERRE SKRIPKINE, ex-ouvrier, ex-membre du Parti; à présent, fiancé.
ZOIA BEREZKINE, ouvrière.
Famille RENAISSANCE :
ELZEVIRE DAVIDOVNA, fiancée, manucure, caissière du salon de coiffure.
ROSALIE PAVLOVNA, mère-coiffeuse.
DAVID OSSIPOVITCH, père-coiffeur.
OLEG BAYAN, autodidacte, ancien propriétaire.


Updating `find_author` & `shortened_lines` accordingly.

---
### Characters
After that, clean characters.

In [382]:
char, no_char = split_by_regex(fnames, REGICES["character"])

with: 889
without: 194


In [525]:
fnames_it = iter(char)
# fnames_it = iter(no_char)

In [534]:
fname = next(fnames_it)
print_file(fname, whole_file=True)

count-1355599-LE_CARROSSE_DU_SAINT.txt
--------------------------------------

LE CARROSSE DU SAINT-SACREMENT

Saynète

de

Prosper Mérimée

Tu veras que mis finezas
Te desenojan.
CALDERÔN
(Cual es la major perfection)

PERSONNAGES :
DON ANCRES DE RIBERA, vice-roi du Pérou .
L'EVEQUE DE LIMA.
LE LICENCIE TOMAS D‘ESQUIVEL.
MARTINEZ, secrétaire intime du vice-roi.
BALTHASAR, valet de chambre du vice-roi.
CAMILA PERICHOLE, comédienne.

La scène est à Lima, en 17...

Le cabinet du Vice-Roi.

Le VICE-ROI, en robe de chambre, assis dans un grand fauteuil auprès d'une table couverte de papiers. Une de ses jambes enveloppée de flanelle repose sur un coussin. Martinez debout auprès de la table, une plume à la main.

MARTINEZ : MM. les Auditeurs attendent la réponse de Votre Altesse.
LE VICE-ROI, d'un ton chagrin : Quelle heure est-il ?
MARTINEZ : Bientôt dix heures. Votre Altesse a justement le temps de s'habiller pour la cérémonie.
LE VICE-ROI : Le temps est beau, dis-tu ?
MARTINEZ : Oui, mons

Updating `find_character` & `shortened_lines`.

In [584]:
fnames_it = iter(fnames)

In [770]:
fname = next(fnames_it)
# fname = "count-959005-LOrphelin_de_la_Chine_-_Voltaire.txt"
# fname = "count-1049195-Arret_36_de_lautobus_40_-_Jean_Sibil.txt"
fname = "count-1541294-Olaf_loriginal.txt"
raw, lines, lines_len = get_lines(fname)
lines_shortened, indices = shortened_lines(fname, lines, lines_len)
print_index_split(lines, indices["author_index"] + indices["char_index"])

Olaf l'original

de

Jean Sibil
****************************************
(Catherine I)
Un salon-salle à manger au mobilier à la fois en harmonie et hétéroclite. Superbes rideaux. Il règne un certain laisser-aller : revues diverses en vrac dans un coin, coussins mal placés sur le divan, fauteuils mal placés (deux sont vis-à-vis pour étendre ses jambes sur l'autre)... Aux murs, très soigneusement accrochés, des photos de mode, des affiches de concerts exceptionnels (toutes musiques), des posters de films récents... Effet de surcharge.
Olaf (la cinquantaine sportive, tenue de footing, une petite queue de cheval et une mèche bleue sur le devant que, debout vers la cheminée, il examine dans une glace de poche) : Je sentais que l'orange me siérait mieux.
Corinne (la trentaine, avachie dans ses fauteuils vis-à-vis, en robe de chambre à grosses fleurs) : Chéri, je t'aime de toutes les couleurs.
Olaf : Tu t'en tires toujours comme ça au lieu de donner des conseils utiles.
Corinne (agressive) : 

---

### Space after "PERSONNAGES :"

In [1619]:
space_after_char = []
for f in fnames:
    raw, lines, lines_len = get_lines(f)
    if regex.search(r"\n\s*PERSONNAGES?\s*:*\n\s*\n", raw):
        space_after_char.append(f)
print(space_after_char)

['count-2158361-ANDROMEDE.txt', 'count-2037607-LHabit_vert.txt', 'count-1182649-LE_MONDE_OU_L.txt', 'count-1037209-LES_MAMELLES_DE_TIRESIAS_-_Guillaume_Apollinaire.txt', 'count-958980-Le_gendre_de_Monsieur_Poirier_-_Emile_Augier.txt']


---

### Checks for results with very few lines

In [1789]:
too_short = []
for f in fnames:
    data = file_data(f, verbose=False)
    dl = data["lines_shortened_len"]
    if dl > 10 and dl < 20:
        too_short.append(data)
len(too_short)

16

In [1790]:
too_short_it = iter(too_short)

In [1807]:
dn = next(too_short_it)
underprint(dn["fname"])
printlines(dn["lines_shortened"])
# printsep('-')
# printlines(dn["lines"])

StopIteration: 

---

# remove lines:
Traduction de

Collaborateur

# avertissement:

count-3141023-Ce_soir_on_improvise.txt

# list of chars without title: 

count-3013840-Menage_moderne.txt

count-2843262-La_grande_voix_de_la_presse.txt